<a href="https://colab.research.google.com/github/barrylunemann/mapping/blob/main/zip_code_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
# A script to create an interactive map with highlighted zip codes
# This would best be used to show service area or a heatmap if you score your
# Zip Codes with that purpose.

# Your Excel file must have a column named "Zip Code" and one named
# "Zip Code Score"

In [54]:
!pip install geopandas
import folium
import pandas as pd
import geopandas as gpd
import branca.colormap as cm
from branca.element import Template, MacroElement

In [55]:
# Use this to mount your Google Drive to the script.
# When this line runs, it will ask for permission to access your drive.
# I will not have access to this, it will only give the session you are in
# access to your drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Here is where you enter your inputs.**

In [ ]:
# Here is where you would enter your inputs.

# Starting Coordinates of the map. I entered the default
# as a zoomed out view of the USA. You can use Google Maps to find the
# center of whatever you would like it centered on.
starting_coordinates = (39.131324555683335, -94.75295974259164)

# How zoomed in do you want it?
zoom_selection = 5

# Import your excel file (reminder you need Zip Code and Zip Code Score)
# You will need to copy the link location to your own Google Drive
main_file = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Data/MN and WI Example.xlsx',
                          sheet_name='Sheet1',
                          dtype={'Zip Code': str})

# Pull in Zip Code geometry data. This is needed to draw the lines on the maps.
geo_zip = gpd.read_file(r"/content/drive/MyDrive/Colab Notebooks/Data/Zip Code Shape Data/cb_2018_us_zcta510_500k.shp")
geo_zip.rename(columns={'ZCTA5CE10': 'Zip Code'}, inplace=True)

# What colors do you want to map your zip code scores? (need to be in HEX)
# This will both be the line color as well as the fill.
# You can add as many numbers as you want, just make sure there is a comma
# between each.
color_fill_dictionary = {1: '#0088D8',
                         2: '#00A82D'}

# How transparent do you want the zip code fill.
# I included this one just in case you want to change the default.
fill_opacity = 0.7

# Tool tip pop-ups on the zip codes. If you have other columns in your Excel
# file that you would like to show up when you click on or hover over a zip code
# this is where you need to list them.
column_list = ['Zip Code',
               'City',
               'Population',
               'Income',
               'State',
               'Market Name']

# Name your output file
output_filename = "Output File"

In [ ]:
main_file.columns

In [ ]:
# Creating the map object
sweet_new_map = folium.Map(starting_coordinates,
                           tiles='openstreetmap',
                           zoom_start=zoom_selection)

In [ ]:
# Merging your zip code list with zip code shape data
main_file = geo_zip.merge(main_file, on='Zip Code', how='left')
main_file = main_file.dropna(subset=['Zip Code Score'])
main_file.fillna('', inplace=True)

In [ ]:
# Creating function to apply the correct colors
style_function = lambda x: {
                            'fillColor': color_fill_dictionary[x['properties']['Zip Code Score']],
                            'color': 'black',
                            'weight': 0.3,
                            'fillOpacity': fill_opacity}

In [ ]:
# Applying the scoring and adding Tooltip and pop-ups.
column_final = column_list.copy()
column_final.append('geometry')
column_final.append('Zip Code Score')
folium.GeoJson(
               main_file.loc[:, column_final],
               style_function=style_function,
               tooltip=folium.GeoJsonTooltip(
                                             fields=column_list,
                                             aliases=column_list,
                  localize=False),
              name='Zip Code Mapping', show=True).add_to(sweet_new_map)
sweet_new_map.add_child(folium.map.LayerControl(collapsed=False))

In [ ]:
#Creating your output file
sweet_new_map.save(output_filename+'.html')
